# 百度网盘AI大赛-图像处理挑战赛：水印智能消除赛第16名方案

[比赛链接](https://aistudio.baidu.com/aistudio/competition/detail/209/0/introduction)

## 比赛介绍
日常生活中带有水印的图片很常见，即使是PS专家，也很难快速且不留痕迹的去除水印。而使用智能去除水印的算法，可以快速自动去除图片中的水印。选手需要通过深度学习技术训练模型，对给定的真实场景下采集得到的带有水印的图片进行处理，并最终输出处理后的扫描结果图片。

本次比赛希望选手结合当下前沿的图像处理技术与计算机视觉技术，提升模型的训练性能和泛化能力，在保证效果精准的同时，注意模型在实际应用中的性能问题，做到尽可能的小而快。

## 技术分析

- 模型选择 

0. 本次模型来源于[摩尔纹消除赛题第二名方案](https://aistudio.baidu.com/aistudio/projectdetail/3439099)，与基线中使用的Unet相比，都是具有Pixel2Pixel形式的网络结构。整体而言，不管是Unet还是IDRnet在本任务上的表象类似。在Resize到512进行预测的预处理下，UNet得分0.58915，IDR得分0.59762。即IDR效果更好，但是两者差距不大。都是PSNR过低。

- 数据增强

0. 从数据量的角度来说，本赛题给出的数据量非常巨大，但是可能由于UNet和IDR都过于单薄，通常在1/10个训练集上已经可以收敛。不需要做额外的数据增强操作。

1. 在预测时，如果将图片缩放到512再预测，会导致图片损失大量信息。但如果不调整图片尺寸，会导致在模型计算过程中，两个隐藏层的输入输出不对应，可以参考[摩尔纹消除赛题第二名方案](https://aistudio.baidu.com/aistudio/projectdetail/3439099)对图片进行补零，从而尽可能贴近原图的尺寸进行预测。在这种处理之下，PSNR大约提升到36。

- loss

0. 赛题的评分标准为ms-ssim和psnr，但实际上大家的ms-ssim的差距较小，主要的差别集中在psnr上。加大损失中psnr的系数，并不能显著对于IDR网络和UNet网络提高分数。

- checkpoint

0. checkpoint保存为model2.pdparams

# 代码
## 解压数据

In [ ]:
! tar -xvf /home/aistudio/data/data142446/bg_images.tar
! tar -xvf /home/aistudio/data/data142446/watermark_datasets.part10.tar

## 构造数据读取器
通过paddle.io.dataset构造读取器，便于读取数据。

数据预处理包括：
1. 将带有水印和不带水印的图片均转化为(3,512,512)的形状
2. 对图片进行归一化

In [3]:
import paddle

import os
import numpy as np
import pandas as pd

import cv2

class MyDateset(paddle.io.Dataset):
    def __init__(self, mode = 'train', watermark_dir = '/home/aistudio/watermark_datasets.part8/', bg_dir = '/home/aistudio/bg_images/'):
        super(MyDateset, self).__init__()

        self.mode = mode 
        self.watermark_dir = watermark_dir
        self.bg_dir = bg_dir

        self.train_list = os.listdir(self.watermark_dir)

    def __getitem__(self, index):
        item = self.train_list[index]
        bg_item = item[:14]+'.jpg'

        img = cv2.imread(self.watermark_dir+item)
        label = cv2.imread(self.bg_dir+bg_item)

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = cv2.cvtColor(label, cv2.COLOR_BGR2RGB)

        img = paddle.vision.transforms.resize(img, (512,512), interpolation='bilinear')
        label = paddle.vision.transforms.resize(label, (512,512), interpolation='bilinear')

        img = img.transpose((2,0,1))
        label = label.transpose((2,0,1))
        
        img = img/255
        label = label/255

        img = paddle.to_tensor(img).astype('float32')
        label = paddle.to_tensor(label).astype('float32')

        return img,label

    def __len__(self):
        return len(self.train_list)

# 对dataloader进行测试
'''
train_dataset=MyDateset()

train_dataloader = paddle.io.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    drop_last=False)

for step, data in enumerate(train_dataloader):
    img, label = data
    print(step, img.shape, label.shape)
    break
'''

'\ntrain_dataset=MyDateset()\n\ntrain_dataloader = paddle.io.DataLoader(\n    train_dataset,\n    batch_size=16,\n    shuffle=True,\n    drop_last=False)\n\nfor step, data in enumerate(train_dataloader):\n    img, label = data\n    print(step, img.shape, label.shape)\n    break\n'

## 定义loss
同样秉承着拿来主义的思想，从[图像评价指标PSNR、SSIM以及MS-SSIM
](https://aistudio.baidu.com/aistudio/projectdetail/1844007?channelType=0&channel=0)复制一份MSSSIM代码过来。

~看不看得懂代码不重要，重要是看得懂文字，明白大佬已经写好了一个现成的直接调用的loss函数~

当然，仅有MSSSIM是不够的，还可以再根据[通过Sub-Pixel实现图像超分辨率](https://www.paddlepaddle.org.cn/documentation/docs/zh/practices/cv/super_resolution_sub_pixel.html#sub-pixel)写一个PSNR的损失函数。

In [4]:
import paddle
import paddle.nn.functional as F

def gaussian1d(window_size, sigma):
    ###window_size = 11
    x = paddle.arange(window_size,dtype='float32')
    x = x - window_size//2
    gauss = paddle.exp(-x ** 2 / float(2 * sigma ** 2))
    # print('gauss.size():', gauss.size())
    ### torch.Size([11])
    return gauss / gauss.sum()

def create_window(window_size, sigma, channel):
    _1D_window = gaussian1d(window_size, sigma).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).unsqueeze(0).unsqueeze(0)
    # print('2d',_2D_window.shape)
    # print(window_size, sigma, channel)
    return _2D_window.expand([channel,1,window_size,window_size])

def _ssim(img1, img2, window, window_size, channel=3 ,data_range = 255.,size_average=True,C=None):
    # size_average for different channel

    padding = window_size // 2

    mu1 = F.conv2d(img1, window, padding=padding, groups=channel)
    mu2 = F.conv2d(img2, window, padding=padding, groups=channel)
    # print(mu1.shape)
    # print(mu1[0,0])
    # print(mu1.mean())
    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2
    sigma1_sq = F.conv2d(img1 * img1, window, padding=padding, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=padding, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=padding, groups=channel) - mu1_mu2
    if C ==None:
        C1 = (0.01*data_range) ** 2
        C2 = (0.03*data_range) ** 2
    else:
        C1 = (C[0]*data_range) ** 2
        C2 = (C[1]*data_range) ** 2
    # l = (2 * mu1_mu2 + C1) / (mu1_sq + mu2_sq + C1)
    # ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    sc = (2 * sigma12 + C2) / (sigma1_sq + sigma2_sq + C2)
    lsc = ((2 * mu1_mu2 + C1) / (mu1_sq + mu2_sq + C1))*sc

    if size_average:
        ### ssim_map.mean()是对这个tensor里面的所有的数值求平均
        return lsc.mean()
    else:
        # ## 返回各个channel的值
        return lsc.flatten(2).mean(-1),sc.flatten(2).mean(-1)

def ms_ssim(
    img1, img2,window, data_range=255, size_average=True, window_size=11, channel=3, sigma=1.5, weights=None, C=(0.01, 0.03)
):

    r""" interface of ms-ssim
    Args:
        img1 (torch.Tensor): a batch of images, (N,C,[T,]H,W)
        img2 (torch.Tensor): a batch of images, (N,C,[T,]H,W)
        data_range (float or int, optional): value range of input images. (usually 1.0 or 255)
        size_average (bool, optional): if size_average=True, ssim of all images will be averaged as a scalar
        win_size: (int, optional): the size of gauss kernel
        win_sigma: (float, optional): sigma of normal distribution
        win (torch.Tensor, optional): 1-D gauss kernel. if None, a new kernel will be created according to win_size and win_sigma
        weights (list, optional): weights for different levels
        K (list or tuple, optional): scalar constants (K1, K2). Try a larger K2 constant (e.g. 0.4) if you get a negative or NaN results.
    Returns:
        torch.Tensor: ms-ssim results
    """
    if not img1.shape == img2.shape:
        raise ValueError("Input images should have the same dimensions.")

    # for d in range(len(img1.shape) - 1, 1, -1):
    #     img1 = img1.squeeze(dim=d)
    #     img2 = img2.squeeze(dim=d)

    if not img1.dtype == img2.dtype:
        raise ValueError("Input images should have the same dtype.")

    if len(img1.shape) == 4:
        avg_pool = F.avg_pool2d
    elif len(img1.shape) == 5:
        avg_pool = F.avg_pool3d
    else:
        raise ValueError(f"Input images should be 4-d or 5-d tensors, but got {img1.shape}")

    smaller_side = min(img1.shape[-2:])

    assert smaller_side > (window_size - 1) * (2 ** 4), "Image size should be larger than %d due to the 4 downsamplings " \
                                                        "with window_size %d in ms-ssim" % ((window_size - 1) * (2 ** 4),window_size)

    if weights is None:
        weights = [0.0448, 0.2856, 0.3001, 0.2363, 0.1333]
    weights = paddle.to_tensor(weights)

    if window is None:
        window = create_window(window_size, sigma, channel)
    assert window.shape == [channel, 1, window_size, window_size], " window.shape error"

    levels = weights.shape[0] # 5
    mcs = []
    for i in range(levels):
        ssim_per_channel, cs =  _ssim(img1, img2, window=window, window_size=window_size,
                                       channel=3, data_range=data_range,C=C, size_average=False)
        if i < levels - 1:
            mcs.append(F.relu(cs))
            padding = [s % 2 for s in img1.shape[2:]]
            img1 = avg_pool(img1, kernel_size=2, padding=padding)
            img2 = avg_pool(img2, kernel_size=2, padding=padding)

    ssim_per_channel = F.relu(ssim_per_channel)  # (batch, channel)
    mcs_and_ssim = paddle.stack(mcs + [ssim_per_channel], axis=0)  # (level, batch, channel) 按照等级堆叠
    ms_ssim_val = paddle.prod(mcs_and_ssim ** weights.reshape([-1, 1, 1]), axis=0) # level 相乘
    print(ms_ssim_val.shape)
    if size_average:
        return ms_ssim_val.mean()
    else:
        # 返回各个channel的值
        return ms_ssim_val.flatten(2).mean(1)


class SSIMLoss(paddle.nn.Layer):
   """
   1. 继承paddle.nn.Layer
   """
   def __init__(self, window_size=11, channel=3, data_range=255., sigma=1.5):
       """
       2. 构造函数根据自己的实际算法需求和使用需求进行参数定义即可
       """
       super(SSIMLoss, self).__init__()
       self.data_range = data_range
       self.C = [0.01, 0.03]
       self.window_size = window_size
       self.channel = channel
       self.sigma = sigma
       self.window = create_window(self.window_size, self.sigma, self.channel)
       # print(self.window_size,self.window.shape)
   def forward(self, input, label):
       """
       3. 实现forward函数，forward在调用时会传递两个参数：input和label
           - input：单个或批次训练数据经过模型前向计算输出结果
           - label：单个或批次训练数据对应的标签数据
           接口返回值是一个Tensor，根据自定义的逻辑加和或计算均值后的损失
       """
       # 使用Paddle中相关API自定义的计算逻辑
       # output = xxxxx
       # return output
       return 1-_ssim(input, label,data_range = self.data_range,
                      window = self.window, window_size=self.window_size, channel=3,
                      size_average=True,C=self.C)

class MS_SSIMLoss(paddle.nn.Layer):
   """
   1. 继承paddle.nn.Layer
   """
   def __init__(self,data_range=255., channel=3, window_size=11, sigma=1.5):
       """
       2. 构造函数根据自己的实际算法需求和使用需求进行参数定义即可
       """
       super(MS_SSIMLoss, self).__init__()
       self.data_range = data_range
       self.C = [0.01, 0.03]
       self.window_size = window_size
       self.channel = channel
       self.sigma = sigma
       self.window = create_window(self.window_size, self.sigma, self.channel)
       # print(self.window_size,self.window.shape)
   def forward(self, input, label):
       """
       3. 实现forward函数，forward在调用时会传递两个参数：input和label
           - input：单个或批次训练数据经过模型前向计算输出结果
           - label：单个或批次训练数据对应的标签数据
           接口返回值是一个Tensor，根据自定义的逻辑加和或计算均值后的损失
       """
       # 使用Paddle中相关API自定义的计算逻辑
       # output = xxxxx
       # return output
       return 1-ms_ssim(input, label, data_range=self.data_range,
                      window = self.window, window_size=self.window_size, channel=self.channel,
                      size_average=True,  sigma=self.sigma,
                      weights=None, C=self.C)

class PSNRLoss(paddle.nn.Layer):
   def __init__(self):
       super(PSNRLoss, self).__init__()

   def forward(self, input, label):
       return 100 - 20 * paddle.log10( ((input - label)**2).mean(axis = [1,2,3])**-0.5 )

## 训练

In [ ]:
# 网络在AIDR_arch中定义，直接导入
from AIDR_arch import AIDR
model = AIDR(num_c=96)
model.train()

train_dataset=MyDateset(watermark_dir = '/home/aistudio/watermark_datasets.part10/')

# 需要接续之前的模型重复训练可以取消注释
param_dict = paddle.load('./model.pdparams')
model.load_dict(param_dict)

train_dataloader = paddle.io.DataLoader(
    train_dataset,
    batch_size=12,
    shuffle=True,
    drop_last=False)

losspsnr = PSNRLoss()
lossfn = SSIMLoss(window_size=3,data_range=1)

max_epoch=20
scheduler = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=0.00001, T_max=max_epoch)
opt = paddle.optimizer.Adam(learning_rate=scheduler, parameters=model.parameters())

now_step=0
for epoch in range(max_epoch):
    for step, data in enumerate(train_dataloader):
        now_step+=1

        img, label = data
        pre = model(img)
        loss1 = lossfn(pre,label).mean()
        loss2 = losspsnr(pre,label).mean()
        loss = (loss1+loss2/50)/2

        loss.backward()
        opt.step()
        opt.clear_gradients()
        if now_step%100==0:
            print("epoch: {}, batch: {}, loss is: {}".format(epoch, step, loss.mean().numpy()))

paddle.save(model.state_dict(), 'model2.pdparams')

## 打包提交






In [ ]:
# 压缩可提交文件
! zip submit.zip model2.pdparams *.py

## 查看预测结果（可选、非常耗时）
是不是想知道自己训练后的网络去除水印之后的图片到底长啥样？直接下载测试集A看看效果吧~

### 下载测试集

In [ ]:
! wget https://staticsns.cdn.bcebos.com/amis/2022-4/1649745356784/watermark_test_datasets.zip
! unzip -oq watermark_test_datasets.zip
! rm -rf watermark_test_datasets.zip

### 在测试集上预测

In [ ]:
! python predict.py watermark_test_datasets/images results

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 